In [1]:
#convolution is to find something useful, and pooling is to bring it together
#think divide and conquer

#for images, convolution might be taking a samller window of the image, and simplfy what it finds in the window, and then shift it
#this should extract features from the image

#pixels -> edges -> lines -> shapes -> features

In [35]:
import time


dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]


            

In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import numpy as np
import time



tensorboard = TensorBoard(log_dir='logs\\%s' %NAME)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
  except RuntimeError as e:
    print(e)

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

#normalize the data
X = X/255.0

#Add input layer


for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            model = Sequential()

            model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            for l in range(conv_layer-1):
                #Deep layers
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten()) #convert out 3D feature maps to 1D feature vectures
          
            for l in range(dense_layer):
              model.add(Dense(layer_size))
              model.add(Activation("relu"))

            #output layer
            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

            X = np.asarray(X)
            y = np.asarray(y)

            model.fit(X, y, batch_size=32, epochs = 1, validation_split=0.1,callbacks=[tensorboard])

Train on 22451 samples, validate on 2495 samples
Epoch 1/10
22451/22451 [==============================] - 83s 4ms/sample - loss: 0.6801 - accuracy: 0.5549 - val_loss: 0.6315 - val_accuracy: 0.6529
Epoch 2/10
 3456/22451 [===>..........................] - ETA: 1:08 - loss: 0.6075 - accuracy: 0.6782

KeyboardInterrupt: 